In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
import logging
import shutil
from omegaconf import OmegaConf
from hydra import initialize, compose
from hydra.core.config_store import ConfigStore
import mlflow
from src.experiments.sner import sner
from pathlib import Path

import os

logging.basicConfig(
    format="%(asctime)s %(levelname)s:%(message)s",
    level=logging.INFO,
    datefmt="%I:%M:%S",
)
logger = logging.getLogger("training")

In [3]:
mlflow.get_tracking_uri()

'https://mlflow-uvl.ifi.uni-heidelberg.de'

In [4]:
sentence_length = 106

run_experiments = os.getenv("UVL_BERT_RUN_EXPERIMENTS", "True") == "True"
pin_commits = os.getenv("UVL_BERT_PIN_COMMITS", "True") == "FALSE"

print(f"{run_experiments=}")
print(f"{pin_commits=}")

run_experiments=True
pin_commits=False


In [5]:
from tooling.config import Experiment, Transformation
#Set Iterations=1 for Split training and testing on datasets. Set dataset to desired dataset split. See tooling.py for split options
base_experiment_config = Experiment(
    name="Base Config 1",
    iterations=1,
    force=False,
    dataset="all",
    lower_case=True,
)

levels_transformation_config = Transformation(
    description="Levels",
    type="Reduced",
    task="Domain_Level",
    domain_data="Domain_Level",
    activity="Domain_Level",
    stakeholder="Domain_Level",
    system_function="Interaction_Level",
    interaction="Interaction_Level",
    interaction_data="Domain_Level",
    workspace="Interaction_Level",
    software="System_Level",
    internal_action="System_Level",
    internal_data="System_Level",
    system_level="System_Level",
)

label_transformation_config = Transformation(
    description="None",
    type="Full",
    task="Task",
    domain_data="Domain_Data",
    activity="Activity",
    stakeholder="Stakeholder",
    system_function="System_Function",
    interaction="Interaction",
    interaction_data="Interaction_Data",
    workspace="Workspace",
    software="System_Level",
    internal_action="System_Level",
    internal_data="System_Level",
    system_level="System_Level",
)

In [6]:
from tooling.transformation import get_hint_transformation
import pickle

hint_transformation = get_hint_transformation(
    transformation_cfg=OmegaConf.structured(levels_transformation_config)
)

Path("./src/service/models/").mkdir(parents=True, exist_ok=True)

hint_label2id = hint_transformation["label2id"]
pickle.dump(
    hint_label2id, open("./src/service/models/hint_label2id.pickle", "wb")
)
hint_id2label = {y: x for x, y in hint_label2id.items()}
pickle.dump(
    hint_id2label, open("./src/service/models/hint_id2label.pickle", "wb")
)

transformation = get_hint_transformation(
    transformation_cfg=OmegaConf.structured(label_transformation_config)
)
label2id = transformation["label2id"]
pickle.dump(label2id, open("./src/service/models/label2id.pickle", "wb"))
id2label = {y: x for x, y in label2id.items()}
pickle.dump(id2label, open("./src/service/models/id2label.pickle", "wb"))

06:09:21 INFO:Hint Label2Id: hint_label2id={'0': 0, 'System_Level': 1, 'Domain_Level': 2, 'Interaction_Level': 3}
06:09:21 INFO:Hint Label2Id: hint_label2id={'0': 0, 'Task': 1, 'Domain_Data': 2, 'Activity': 3, 'Stakeholder': 4, 'System_Function': 5, 'Interaction': 6, 'Interaction_Data': 7, 'Workspace': 8, 'System_Level': 9}


## Train BiLSTM First Stage Model


In [9]:
'''
from tooling.observability import get_run_id
from tooling.config import BiLSTMConfig, BiLSTM

from copy import deepcopy

bilstm_experiment_config = deepcopy(base_experiment_config)
bilstm_experiment_config.name = "Production"

bilstm_config = BiLSTM(
    batch_size=16,
    learning_rate=0.006,
    number_epochs=5,
    weighted_classes=False,
    sentence_length=106,
)

bilstm_cfg: BiLSTMConfig = OmegaConf.structured(
    BiLSTMConfig(
        bilstm=bilstm_config,
        experiment=bilstm_experiment_config,
        transformation=levels_transformation_config,
    )
)

# bilstm_cfg.experiment.force = True

if run_experiments:
    from experiments.bilstm import bilstm

    bilstm(bilstm_cfg)


bilstm_run_id = get_run_id(bilstm_cfg, pin_commit=pin_commits)

print(bilstm_run_id)
print(mlflow.get_artifact_uri())

bilstm_run = mlflow.get_run(bilstm_run_id)
mlflow.artifacts.download_artifacts(
    f"{bilstm_run.info.artifact_uri}/0_model",
    dst_path=Path("./src/service/models/"),
)
try:
    shutil.rmtree(Path("./src/service/models/bilstm"))
except FileNotFoundError:
    pass
Path("./src/service/models/0_model").rename("./src/service/models/bilstm")
'''

SyntaxError: unterminated triple-quoted string literal (detected at line 48) (87726336.py, line 1)

## Train SNER First Stage Model


In [13]:
'''
from tooling.observability import (get_run_id)
from tooling.config import SNERConfig, SNER

from copy import deepcopy

sner_experiment_config = deepcopy(base_experiment_config)
sner_experiment_config.name = "Production"

sner_config = SNER()

sner_cfg = OmegaConf.structured(
    SNERConfig(
        sner=sner_config,
        experiment=sner_experiment_config,
        transformation=levels_transformation_config,
    )
)

if run_experiments:
    from experiments.sner import sner

    sner(OmegaConf.create(sner_cfg))

sner_run_id = get_run_id(sner_cfg, pin_commit=pin_commits)
print(mlflow.get_artifact_uri())
print(sner_run_id)

sner_run = mlflow.get_run(sner_run_id)
mlflow.artifacts.download_artifacts(
    f"{sner_run.info.artifact_uri}/0_model.ser.gz",
    dst_path=Path("./src/service/models/"),
)
try:
    Path("./src/service/models/sner.ser.gz").unlink()
except FileNotFoundError:
    pass
Path("./src/service/models/0_model.ser.gz").rename(
    "./src/service/models/sner.ser.gz"
)
'''

SyntaxError: unterminated triple-quoted string literal (detected at line 40) (2848408053.py, line 1)

## Train BERT First Stage Model


In [14]:
'''
from tooling.observability import get_run_id
from tooling.config import BERTConfig, BERT

from copy import deepcopy

bert_1_experiment_config = deepcopy(base_experiment_config)
bert_1_experiment_config.name = "Production"
bert_1_experiment_config.force = False

bert_1_config = BERT(
    max_len=123,
    number_epochs=11,
    train_batch_size=8,
    weight_decay=0.01,
    weighted_classes=True,
    learning_rate_bert=3e-05,
    learning_rate_classifier=0.0005,
    validation_batch_size=64,
)


bert_1_cfg = OmegaConf.structured(
    BERTConfig(
        bert=bert_1_config,
        experiment=bert_1_experiment_config,
        transformation=levels_transformation_config,
    )
)

if run_experiments:
    from experiments.bert import bert

    bert(OmegaConf.create(bert_1_cfg))

bert_1_run_id = get_run_id(bert_1_cfg, pin_commit=pin_commits)

print(bert_1_run_id)

bert_1_run = mlflow.get_run(bert_1_run_id)
mlflow.artifacts.download_artifacts(
    f"{bert_1_run.info.artifact_uri}/0_model",
    dst_path=Path("./src/service/models/"),
)
try:
    shutil.rmtree(Path("./src/service/models/bert_1"))
except FileNotFoundError:
    pass
Path("./src/service/models/0_model").rename("./src/service/models/bert_1")
## Train BERT Second Stage Model for BERT First Stage
'''

02:26:05 INFO:
bert:
  model: bert-large-uncased
  type: BERT
  max_len: 123
  train_batch_size: 8
  validation_batch_size: 64
  number_epochs: 11
  learning_rate_bert: 3.0e-05
  learning_rate_classifier: 0.0005
  weight_decay: 0.01
  weighted_classes: true
experiment:
  name: Production
  description: ''
  random_state: 125
  folds: 5
  iterations: 1
  average: macro
  dataset: all
  lower_case: true
  force: false
  pin_commit: false
  smote: false
  smote_k_neighbors: 5
  smote_sampling_strategy: not majority
  smote_balance_to_average: false
transformation:
  description: Levels
  type: Reduced
  task: Domain_Level
  goals: null
  domain_data: Domain_Level
  activity: Domain_Level
  stakeholder: Domain_Level
  system_function: Interaction_Level
  interaction: Interaction_Level
  interaction_data: Domain_Level
  workspace: Interaction_Level
  software: System_Level
  internal_action: System_Level
  internal_data: System_Level
  system_level: System_Level

02:26:05 INFO:New experimen

Map:   0%|          | 0/2488 [00:00<?, ? examples/s]

Map:   0%|          | 0/622 [00:00<?, ? examples/s]

E:\BERT\uvl-tore-classifier-bert\venv310\lib\site-packages\transformers\training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
E:\BERT\uvl-tore-classifier-bert\venv310\lib\site-packages\transformers\models\bert\modeling_bert.py:435: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
02:27:27 INFO:Logged iteration result res.precision=0.6903478382818348 res.recall=0.8199129858857586


{'eval_loss': 0.5139986276626587, 'eval_step': 0, 'eval_precision': 0.6903478382818348, 'eval_recall': 0.8199129858857586, 'eval_f1': 0.749572720589389, 'eval_label_count': 4, 'eval_runtime': 5.1846, 'eval_samples_per_second': 119.97, 'eval_steps_per_second': 1.929, 'epoch': 1.0}
{'loss': 0.5443, 'grad_norm': 8.23720645904541, 'learning_rate': 2.561531715872552e-05, 'epoch': 1.607717041800643}


...

{'eval_loss': 1.660742998123169, 'eval_step': 18, 'eval_precision': 0.8093775536293907, 'eval_recall': 0.8117250397806778, 'eval_f1': 0.810549597031228, 'eval_label_count': 4, 'eval_runtime': 4.5929, 'eval_samples_per_second': 135.426, 'eval_steps_per_second': 2.177, 'epoch': 10.0}


02:44:23 INFO:Logged iteration result res.precision=0.8050577289903632 res.recall=0.8133762651366054


{'eval_loss': 1.6525459289550781, 'eval_step': 20, 'eval_precision': 0.8050577289903632, 'eval_recall': 0.8133762651366054, 'eval_f1': 0.8091956189770539, 'eval_label_count': 4, 'eval_runtime': 4.5014, 'eval_samples_per_second': 138.18, 'eval_steps_per_second': 2.222, 'epoch': 11.0}
{'train_runtime': 1118.9819, 'train_samples_per_second': 24.458, 'train_steps_per_second': 3.057, 'train_loss': 0.1395734454832102, 'epoch': 11.0}


02:45:22 INFO:Logged iteration result res.precision=0.8050577289903632 res.recall=0.8133762651366054
02:45:23 INFO:Logged iteration result res.precision=0.8050577289903632 res.recall=0.8133762651366054
02:45:23 INFO:Finished iteration=0
02:45:23 INFO:Logging model artifact (might take a while)
02:45:55 WARNING:Retrying (Retry(total=4, connect=5, read=5, redirect=5, status=5)) after connection broken by 'SSLEOFError(8, 'EOF occurred in violation of protocol (_ssl.c:2384)')': /api/2.0/mlflow-artifacts/artifacts/6/2fd7eb8bb98240a2bc79aaa6986e4487/artifacts/0_model/model.safetensors
02:46:31 WARNING:Retrying (Retry(total=3, connect=5, read=5, redirect=5, status=5)) after connection broken by 'SSLEOFError(8, 'EOF occurred in violation of protocol (_ssl.c:2384)')': /api/2.0/mlflow-artifacts/artifacts/6/2fd7eb8bb98240a2bc79aaa6986e4487/artifacts/0_model/model.safetensors
02:47:11 WARNING:Retrying (Retry(total=2, connect=5, read=5, redirect=5, status=5)) after connection broken by 'SSLEOFError

MlflowException: API request to https://mlflow-uvl.ifi.uni-heidelberg.de/api/2.0/mlflow-artifacts/artifacts/6/2fd7eb8bb98240a2bc79aaa6986e4487/artifacts/0_model/model.safetensors failed with exception HTTPSConnectionPool(host='mlflow-uvl.ifi.uni-heidelberg.de', port=443): Max retries exceeded with url: /api/2.0/mlflow-artifacts/artifacts/6/2fd7eb8bb98240a2bc79aaa6986e4487/artifacts/0_model/model.safetensors (Caused by SSLError(SSLEOFError(8, 'EOF occurred in violation of protocol (_ssl.c:2384)')))

In [15]:
'''
from tooling.observability import get_run_id
from tooling.config import DualModelStagedBERTConfig, StagedBERT

from copy import deepcopy

bert_2_bert_experiment_config = deepcopy(base_experiment_config)
bert_2_bert_experiment_config.name = "Production"

bert_2_bert_config = StagedBERT(
    max_len=123,
    layers=[],
    learning_rate_bert=3e-05,
    learning_rate_classifier=0.16,
    number_epochs=8,
    train_batch_size=8,
    weight_decay=0.1,
    weighted_classes=True,
    validation_batch_size=64,
)

cfg = DualModelStagedBERTConfig(
    bert=bert_2_bert_config,
    experiment=bert_2_bert_experiment_config,
    transformation=label_transformation_config,
    first_model_bert=bert_1_cfg,
)

bert_2_bert_cfg: DualModelStagedBERTConfig = OmegaConf.structured(cfg)

if run_experiments:
    from experiments.dual_model_staged_bert import dual_stage_bert

    dual_stage_bert(OmegaConf.create(bert_2_bert_cfg))

bert_2_bert_run_id = get_run_id(bert_2_bert_cfg, pin_commit=pin_commits)

print(bert_2_bert_run_id)

bert_2_bert_run = mlflow.get_run(bert_2_bert_run_id)
mlflow.artifacts.download_artifacts(
    f"{bert_2_bert_run.info.artifact_uri}/0_model",
    dst_path=Path("./src/service/models/"),
)
try:
    shutil.rmtree(Path("./src/service/models/bert_2_bert"))
except FileNotFoundError:
    pass
Path("./src/service/models/0_model").rename("./src/service/models/bert_2_bert")
'''

02:49:22 INFO:
first_model_bert:
  bert:
    model: bert-large-uncased
    type: BERT
    max_len: 123
    train_batch_size: 8
    validation_batch_size: 64
    number_epochs: 11
    learning_rate_bert: 3.0e-05
    learning_rate_classifier: 0.0005
    weight_decay: 0.01
    weighted_classes: true
  experiment:
    name: Production
    description: ''
    random_state: 125
    folds: 5
    iterations: 1
    average: macro
    dataset: all
    lower_case: true
    force: false
    pin_commit: false
    smote: false
    smote_k_neighbors: 5
    smote_sampling_strategy: not majority
    smote_balance_to_average: false
  transformation:
    description: Levels
    type: Reduced
    task: Domain_Level
    goals: null
    domain_data: Domain_Level
    activity: Domain_Level
    stakeholder: Domain_Level
    system_function: Interaction_Level
    interaction: Interaction_Level
    interaction_data: Domain_Level
    workspace: Interaction_Level
    software: System_Level
    internal_action: Sy

Map:   0%|          | 0/2488 [00:00<?, ? examples/s]

Map:   0%|          | 0/622 [00:00<?, ? examples/s]

E:\BERT\uvl-tore-classifier-bert\venv310\lib\site-packages\transformers\training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
02:51:52 INFO:Logged iteration result res.precision=0.6819089064097362 res.recall=0.8139693008422187


{'eval_loss': 0.5252214670181274, 'eval_step': 0, 'eval_precision': 0.6819089064097362, 'eval_recall': 0.8139693008422187, 'eval_f1': 0.7421097694953261, 'eval_label_count': 4, 'eval_runtime': 30.8961, 'eval_samples_per_second': 20.132, 'eval_steps_per_second': 0.324, 'epoch': 1.0}
{'loss': 0.5513, 'grad_norm': 6.163368225097656, 'learning_rate': 2.561531715872552e-05, 'epoch': 1.607717041800643}


...

{'eval_loss': 1.6771281957626343, 'eval_step': 18, 'eval_precision': 0.8018761819997918, 'eval_recall': 0.8040091055955558, 'eval_f1': 0.8029412273319085, 'eval_label_count': 4, 'eval_runtime': 10.7302, 'eval_samples_per_second': 57.967, 'eval_steps_per_second': 0.932, 'epoch': 10.0}


03:26:36 INFO:Logged iteration result res.precision=0.8044535315969952 res.recall=0.8009834272072174


{'eval_loss': 1.7390738725662231, 'eval_step': 20, 'eval_precision': 0.8044535315969952, 'eval_recall': 0.8009834272072174, 'eval_f1': 0.8027147291382264, 'eval_label_count': 4, 'eval_runtime': 29.1447, 'eval_samples_per_second': 21.342, 'eval_steps_per_second': 0.343, 'epoch': 11.0}
{'train_runtime': 2244.684, 'train_samples_per_second': 12.192, 'train_steps_per_second': 1.524, 'train_loss': 0.14421657963964332, 'epoch': 11.0}


03:28:12 INFO:Logged iteration result res.precision=0.8044535315969952 res.recall=0.8009834272072174
03:28:14 INFO:Logged iteration result res.precision=0.8044535315969952 res.recall=0.8009834272072174
03:28:14 INFO:Finished iteration=0
03:28:14 INFO:Logging model artifact (might take a while)
03:28:44 WARNING:Retrying (Retry(total=4, connect=5, read=5, redirect=5, status=5)) after connection broken by 'SSLEOFError(8, 'EOF occurred in violation of protocol (_ssl.c:2384)')': /api/2.0/mlflow-artifacts/artifacts/6/ea07e5979b2142128114559b5c743eb4/artifacts/0_model/model.safetensors
03:29:21 WARNING:Retrying (Retry(total=3, connect=5, read=5, redirect=5, status=5)) after connection broken by 'SSLEOFError(8, 'EOF occurred in violation of protocol (_ssl.c:2384)')': /api/2.0/mlflow-artifacts/artifacts/6/ea07e5979b2142128114559b5c743eb4/artifacts/0_model/model.safetensors
03:30:00 WARNING:Retrying (Retry(total=2, connect=5, read=5, redirect=5, status=5)) after connection broken by 'SSLEOFError

MlflowException: API request to https://mlflow-uvl.ifi.uni-heidelberg.de/api/2.0/mlflow-artifacts/artifacts/6/ea07e5979b2142128114559b5c743eb4/artifacts/0_model/model.safetensors failed with exception HTTPSConnectionPool(host='mlflow-uvl.ifi.uni-heidelberg.de', port=443): Max retries exceeded with url: /api/2.0/mlflow-artifacts/artifacts/6/ea07e5979b2142128114559b5c743eb4/artifacts/0_model/model.safetensors (Caused by SSLError(SSLEOFError(8, 'EOF occurred in violation of protocol (_ssl.c:2384)')))

## Train BERT Second Stage Model for SNER First Stage


In [ ]:
'''
from tooling.observability import get_run_id
from tooling.config import DualModelStagedBERTConfig, StagedBERT

from copy import deepcopy

import os

os.environ["TOKENIZERS_PARALLELISM"] = "true"

bert_2_sner_experiment_config = deepcopy(base_experiment_config)
bert_2_sner_experiment_config.name = "Production"

bert_2_sner_config = StagedBERT(
    max_len=123,
    layers=[],
    learning_rate_bert=3e-05,
    learning_rate_classifier=0.16,
    number_epochs=5,
    train_batch_size=8,
    weight_decay=0.1,
    weighted_classes=True,
)

bert_2_sner_cfg = OmegaConf.structured(
    DualModelStagedBERTConfig(
        bert=bert_2_sner_config,
        experiment=bert_2_sner_experiment_config,
        transformation=label_transformation_config,
        first_model_sner=sner_cfg,
    )
)

if run_experiments:
    from experiments.dual_model_staged_bert import dual_stage_bert

    dual_stage_bert(OmegaConf.create(bert_2_sner_cfg))

bert_2_sner_run_id = get_run_id(bert_2_sner_cfg, pin_commit=pin_commits)

print(bert_2_sner_run_id)

bert_2_sner_run = mlflow.get_run(bert_2_sner_run_id)
mlflow.artifacts.download_artifacts(
    f"{bert_2_sner_run.info.artifact_uri}/0_model",
    dst_path=Path("./src/service/models/"),
)
try:
    shutil.rmtree(Path("./src/service/models/bert_2_sner"))
except FileNotFoundError:
    pass

Path("./src/service/models/0_model").rename("./src/service/models/bert_2_sner")
'''

## Train BERT Second Stage Model for BILSTM First Stage


In [ ]:
'''
from tooling.observability import get_run_id
from tooling.config import DualModelStagedBERTConfig, StagedBERT

from copy import deepcopy

bilstm_config = BiLSTM(
    batch_size=16,
    learning_rate=0.006,
    number_epochs=5,
    weighted_classes=False,
    sentence_length=106,
)


bert_2_bilstm_experiment_config = deepcopy(base_experiment_config)
bert_2_bilstm_experiment_config.name = "Production"

bert_2_bilstm_config = StagedBERT(
    max_len=123,
    layers=[],
    learning_rate_bert=3e-05,
    learning_rate_classifier=0.16,
    number_epochs=5,
    train_batch_size=8,
    weight_decay=0.1,
    weighted_classes=True,
)

bert_2_bilstm_cfg = OmegaConf.structured(
    DualModelStagedBERTConfig(
        bert=bert_2_bilstm_config,
        experiment=bert_2_bilstm_experiment_config,
        transformation=label_transformation_config,
        first_model_bilstm=bilstm_cfg,
    )
)
bert_2_bilstm_cfg.first_model_bilstm.bilstm.sentence_length = 106


if run_experiments:
    from experiments.dual_model_staged_bert import dual_stage_bert

    dual_stage_bert(OmegaConf.create(bert_2_bilstm_cfg))

bert_2_bilstm_run_id = get_run_id(bert_2_bilstm_cfg, pin_commit=pin_commits)

print(bert_2_bilstm_run_id)

bert_2_bilstm_run = mlflow.get_run(bert_2_bilstm_run_id)
mlflow.artifacts.download_artifacts(
    f"{bert_2_bilstm_run.info.artifact_uri}/0_model",
    dst_path=Path("./src/service/models/"),
)
try:
    shutil.rmtree(Path("./src/service/models/bert_2_bilstm"))
except FileNotFoundError:
    pass
Path("./src/service/models/0_model").rename(
    "./src/service/models/bert_2_bilstm"
)
'''

## Train BERT E2E Model


In [7]:
from tooling.observability import get_run_id
from tooling.config import BERTConfig, BERT

from copy import deepcopy

bert_experiment_config = deepcopy(base_experiment_config)
bert_experiment_config.name = "Production"

bert_config = BERT(
    learning_rate_bert=6e-05,
    learning_rate_classifier=0.1,
    max_len=123,
    number_epochs=1 ,
    train_batch_size=8,
    weight_decay=0.01,
    weighted_classes=False,
)

bert_cfg = OmegaConf.structured(
    BERTConfig(
        bert=bert_config,
        experiment=bert_experiment_config,
        transformation=label_transformation_config,
    )
)

if run_experiments:
    from experiments.bert import bert

    bert(OmegaConf.create(bert_cfg))

bert_run_id = get_run_id(bert_cfg, pin_commit=pin_commits)

print(bert_run_id)

run = mlflow.get_run(bert_run_id)
mlflow.artifacts.download_artifacts(
    f"{run.info.artifact_uri}/0_model", dst_path=Path("./src/service/models/")
)
try:
    shutil.rmtree(Path("./src/service/models/bert"))
except FileNotFoundError:
    pass
Path("./src/service/models/0_model").rename("./src/service/models/bert")

06:09:31 INFO:PyTorch version 2.3.0+cu118 available.
06:09:31 INFO:TensorFlow version 2.15.1 available.
06:09:35 WARNING:From E:\BERT\uvl-tore-classifier-bert\venv310\lib\site-packages\keras\src\losses.py:2976: The name tf.losses.sparse_softmax_cross_entropy is deprecated. Please use tf.compat.v1.losses.sparse_softmax_cross_entropy instead.

06:09:35 INFO:
bert:
  model: bert-large-uncased
  type: BERT
  max_len: 123
  train_batch_size: 8
  validation_batch_size: 32
  number_epochs: 1
  learning_rate_bert: 6.0e-05
  learning_rate_classifier: 0.1
  weight_decay: 0.01
  weighted_classes: false
experiment:
  name: Production
  description: ''
  random_state: 125
  folds: 5
  iterations: 1
  average: macro
  dataset: all
  lower_case: true
  force: false
  pin_commit: false
  smote: false
  smote_k_neighbors: 5
  smote_sampling_strategy: not majority
  smote_balance_to_average: false
transformation:
  description: None
  type: Full
  task: Task
  goals: null
  domain_data: Domain_Data
  ac

MlflowException: API request to https://mlflow-uvl.ifi.uni-heidelberg.de/api/2.0/mlflow-artifacts/artifacts/6/9f0a9d8307da45f29d017f024a5348c4/artifacts/imported_dataset.csv failed with exception HTTPSConnectionPool(host='mlflow-uvl.ifi.uni-heidelberg.de', port=443): Max retries exceeded with url: /api/2.0/mlflow-artifacts/artifacts/6/9f0a9d8307da45f29d017f024a5348c4/artifacts/imported_dataset.csv (Caused by ResponseError('too many 500 error responses'))

In [ ]:
##Train RoBERTa E2E Model


In [14]:
'''
from tooling.observability import get_run_id
from tooling.config import RoBERTaConfig, RoBERTa

from copy import deepcopy

roberta_experiment_config = deepcopy(base_experiment_config)
roberta_experiment_config.name = "Production"

roberta_config = RoBERTa(
    learning_rate_roberta=6e-05,
    learning_rate_classifier=0.1,
    max_len=123,
    number_epochs=11,
    train_batch_size=8,
    weight_decay=0.01,
    weighted_classes=False,
)

roberta_cfg = OmegaConf.structured(
    RoBERTaConfig(
        roberta=roberta_config,
        experiment=roberta_experiment_config,
        transformation=label_transformation_config,
    )
)

if run_experiments:
    from experiments.roberta import roberta

    roberta(OmegaConf.create(roberta_cfg))

roberta_run_id = get_run_id(roberta_cfg, pin_commit=pin_commits)

print(roberta_run_id)

run = mlflow.get_run(roberta_run_id)
mlflow.artifacts.download_artifacts(
    f"{run.info.artifact_uri}/0_model", dst_path=Path("./src/service/models/")
)
try:
    shutil.rmtree(Path("./src/service/models/roberta"))
except FileNotFoundError:
    pass
Path("./src/service/models/0_model").rename("./src/service/models/roberta")
'''

01:44:16 INFO:
roberta:
  model: Jean-Baptiste/roberta-large-ner-english
  type: RoBERTa
  max_len: 123
  train_batch_size: 8
  validation_batch_size: 32
  number_epochs: 11
  learning_rate_roberta: 6.0e-05
  learning_rate_classifier: 0.1
  weight_decay: 0.01
  weighted_classes: false
experiment:
  name: Production
  description: ''
  random_state: 125
  folds: 5
  iterations: 1
  average: macro
  dataset: all
  lower_case: true
  force: false
  pin_commit: false
  smote: false
  smote_k_neighbors: 5
  smote_sampling_strategy: not majority
transformation:
  description: None
  type: Full
  task: Task
  goals: null
  domain_data: Domain_Data
  activity: Activity
  stakeholder: Stakeholder
  system_function: System_Function
  interaction: Interaction
  interaction_data: Interaction_Data
  workspace: Workspace
  software: System_Level
  internal_action: System_Level
  internal_data: System_Level
  system_level: System_Level

01:44:16 INFO:New experiment. Running
01:44:16 INFO:Entering mlf

Map:   0%|          | 0/2488 [00:00<?, ? examples/s]

Map:   0%|          | 0/622 [00:00<?, ? examples/s]

E:\BERT\uvl-tore-classifier-bert\venv310\lib\site-packages\transformers\training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
01:45:21 INFO:Logged iteration result res.precision=0.5793384603014975 res.recall=0.5294931132606071


{'eval_loss': 0.606141209602356, 'eval_step': 0, 'eval_precision': 0.5793384603014975, 'eval_recall': 0.5294931132606071, 'eval_f1': 0.5532954369096804, 'eval_label_count': 10, 'eval_runtime': 3.4457, 'eval_samples_per_second': 180.515, 'eval_steps_per_second': 5.804, 'epoch': 1.0}
{'loss': 1.2792, 'grad_norm': 15.624954223632812, 'learning_rate': 5.123063431745104e-05, 'epoch': 1.607717041800643}


...

02:01:44 INFO:Logged iteration result res.precision=0.6904512276980231 res.recall=0.6926464362913085


{'eval_loss': 0.9886234998703003, 'eval_step': 20, 'eval_precision': 0.6904512276980231, 'eval_recall': 0.6926464362913085, 'eval_f1': 0.6915470899120589, 'eval_label_count': 10, 'eval_runtime': 3.5215, 'eval_samples_per_second': 176.631, 'eval_steps_per_second': 5.679, 'epoch': 11.0}
{'train_runtime': 1083.2292, 'train_samples_per_second': 25.265, 'train_steps_per_second': 3.158, 'train_loss': 0.45614679554546483, 'epoch': 11.0}


02:02:35 INFO:Logged iteration result res.precision=0.6904512276980231 res.recall=0.6926464362913085
02:02:35 INFO:Logged iteration result res.precision=0.6904512276980231 res.recall=0.6926464362913085
02:02:35 INFO:Finished iteration=0
02:02:35 INFO:Logging model artifact (might take a while)
02:02:38 INFO:Breaking early after iteration=0 of 5 folds
02:02:38 INFO:Logged experiment result res.mean_precision=0.6904512276980231 res.mean_recall=0.6926464362913085
02:02:39 INFO:Left mlflow context


73c3721d2c3e41e6b0539148725132df


WindowsPath('src/service/models/roberta')